In [2]:
import time
import datetime
import sys
import numpy as np
from TimeTagger import Coincidences, createTimeTagger, freeTimeTagger, Countrate, CoincidenceTimestamp
import matplotlib.pyplot as plt

coin_lst = [
    [1, 3, 5, 7, 9, 11], [2, 3, 5, 7, 9, 11], [1, 4, 5, 7, 9, 11], [2, 4, 5, 7, 9, 11], 
    [1, 3, 6, 7, 9, 11], [2, 3, 6, 7, 9, 11], [1, 4, 6, 7, 9, 11], [2, 4, 6, 7, 9, 11], 
    [1, 3, 5, 8, 9, 11], [2, 3, 5, 8, 9, 11], [1, 4, 5, 8, 9, 11], [2, 4, 5, 8, 9, 11], 
    [1, 3, 6, 8, 9, 11], [2, 3, 6, 8, 9, 11], [1, 4, 6, 8, 9, 11], [2, 4, 6, 8, 9, 11], 
    [1, 3, 5, 7, 10, 11], [2, 3, 5, 7, 10, 11], [1, 4, 5, 7, 10, 11], [2, 4, 5, 7, 10, 11], 
    [1, 3, 6, 7, 10, 11], [2, 3, 6, 7, 10, 11], [1, 4, 6, 7, 10, 11], [2, 4, 6, 7, 10, 11], 
    [1, 3, 5, 8, 10, 11], [2, 3, 5, 8, 10, 11], [1, 4, 5, 8, 10, 11], [2, 4, 5, 8, 10, 11], 
    [1, 3, 6, 8, 10, 11], [2, 3, 6, 8, 10, 11], [1, 4, 6, 8, 10, 11], [2, 4, 6, 8, 10, 11], 
    [1, 3, 5, 7, 9, 12], [2, 3, 5, 7, 9, 12], [1, 4, 5, 7, 9, 12], [2, 4, 5, 7, 9, 12], 
    [1, 3, 6, 7, 9, 12], [2, 3, 6, 7, 9, 12], [1, 4, 6, 7, 9, 12], [2, 4, 6, 7, 9, 12], 
    [1, 3, 5, 8, 9, 12], [2, 3, 5, 8, 9, 12], [1, 4, 5, 8, 9, 12], [2, 4, 5, 8, 9, 12], 
    [1, 3, 6, 8, 9, 12], [2, 3, 6, 8, 9, 12], [1, 4, 6, 8, 9, 12], [2, 4, 6, 8, 9, 12], 
    [1, 3, 5, 7, 10, 12], [2, 3, 5, 7, 10, 12], [1, 4, 5, 7, 10, 12], [2, 4, 5, 7, 10, 12], 
    [1, 3, 6, 7, 10, 12], [2, 3, 6, 7, 10, 12], [1, 4, 6, 7, 10, 12], [2, 4, 6, 7, 10, 12], 
    [1, 3, 5, 8, 10, 12], [2, 3, 5, 8, 10, 12], [1, 4, 5, 8, 10, 12], [2, 4, 5, 8, 10, 12], 
    [1, 3, 6, 8, 10, 12], [2, 3, 6, 8, 10, 12], [1, 4, 6, 8, 10, 12], [2, 4, 6, 8, 10, 12]
]

# init tagger

tagger = createTimeTagger()
for i in range(1, 17):
    tagger.setTriggerLevel(i, 0.8)

# set delay

tagger.setInputDelay(2, -2652)
tagger.setInputDelay(3, -6072)
tagger.setInputDelay(4, -9302)
tagger.setInputDelay(5, -4282)
tagger.setInputDelay(6, -8932)
tagger.setInputDelay(7, -4477)
tagger.setInputDelay(8, -8962)
tagger.setInputDelay(9, -8657)
tagger.setInputDelay(10, -7857)
tagger.setInputDelay(11, -2867)
tagger.setInputDelay(12, -12042)

# create channels

coin = Coincidences(
                tagger, coin_lst, coincidenceWindow=5000, timestamp=CoincidenceTimestamp.Average
                )

countrate = Countrate(tagger=tagger, channels=coin.getChannels())


t = datetime.datetime.now()
start_time = t.strftime("%y.%m.%d_%H.%M.%S")
filename = f"data_tomo/tomo_{start_time}.txt"

# TOMO

In [3]:
sec = 3600 # 90 min
loop = 3
estimated_finish_time = (t + datetime.timedelta(seconds=sec*loop)).strftime("20%y-%m-%d %H:%M:%S")
print(f"Estimated finish time : {estimated_finish_time}")
f = open(filename, "w")

try:
    for l in range(loop):
        countrate.startFor(int(sec*1e12))
        for i in range(0, sec):
            time.sleep(1)
            count = countrate.getCountsTotal()
            sys.stdout.write(f"Loop {l + 1}, Time remaining: {(sec - i):4d} s, total counts = {sum(count)}, counts/min={sum(count)*60/(i+1):.2f}, {i/(sum(count)+1):.2f} sec/count \n")
            sys.stdout.write(f"{count}\n")
            sys.stdout.flush()
            sys.stdout.write("\x1b[1A\x1b[2K\x1b[1A\x1b[2K\x1b[1A\x1b[2K")
        print("")
        countrate.waitUntilFinished()
        count = countrate.getCountsTotal()
        f.write(f"{count}\n")
        f.flush()
finally:
    if countrate.isRunning():
        countrate.stop()
        count = countrate.getCountsTotal()
        f.write(f"{count}\n")
    f.close()
    freeTimeTagger(tagger=tagger)

Estimated finish time : 2024-04-05 22:56:49
Loop 1, Time remaining: 3600 s, total counts = 0, counts/min=0.00, 0.00 seconds per countn[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Loop 1, Time remaining: 3599 s, total counts = 0, counts/min=0.00, 1.00 seconds per countn[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Loop 1, Time remaining: 3598 s, total counts = 0, counts/min=0.00, 2.00 seconds per countn[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Loop 1, Time remaining: 3597 s, total counts = 0, counts/min=0.00, 3.00 seconds per countn[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Loop 1, Time remaining: 3596 s, total counts = 0, counts/min=0.00, 4.00 

KeyboardInterrupt: 

In [16]:
freeTimeTagger(tagger=tagger)

False